Import libraries for this task.

In [1]:
import cv2
import time
from datetime import timedelta
import math
import numpy as np

File path in strings, as well as loading the file into cv2.VideoCapture().

In [2]:
filepath1 = 'Traffic_Laramie_1.mp4'
filepath2 = 'Traffic_Laramie_2.mp4'

video=cv2.VideoCapture('Traffic_Laramie_1.mp4')
video2= cv2.VideoCapture('Traffic_Laramie_2.mp4')

Data/Information about the two videos.

In [3]:
frames_count, fps, width, height = video.get(cv2.CAP_PROP_FRAME_COUNT), video.get(cv2.CAP_PROP_FPS), video.get(cv2.CAP_PROP_FRAME_WIDTH), video.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = int(width)
height = int(height)
length = frames_count/fps
minutes = int(length/60)
seconds = int(length%60)
duration1 = str(minutes) + ':' + str(seconds)

print(filepath1)
print(f'Video Length:{duration1}')
print(f'Total frames: {frames_count}')
print(f'Fps: {fps}')
print(f'Video Width: {width}')
print(f'Video Height: {height}')

print('\n')

frames_count, fps, width, height = video2.get(cv2.CAP_PROP_FRAME_COUNT), video.get(cv2.CAP_PROP_FPS), video.get(cv2.CAP_PROP_FRAME_WIDTH), video.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = int(width)
height = int(height)
length = frames_count/fps
minutes = int(length/60)
seconds = int(length%60)
duration2 = str(minutes) + ':' + str(seconds)

print(filepath2)
print(f'Video Length:{duration2}')
print(f'Total frames: {frames_count}')
print(f'Fps: {fps}')
print(f'Video Width: {width}')
print(f'Video Height: {height}')

Traffic_Laramie_1.mp4
Video Length:2:57
Total frames: 4448.0
Fps: 25.0
Video Width: 1040
Video Height: 600


Traffic_Laramie_2.mp4
Video Length:1:45
Total frames: 2642.0
Fps: 25.0
Video Width: 1040
Video Height: 600


### The Tracking Algorithm

Below is the class that will be utilized to track and count the number of cars that go from the city's downtown to the city centre. It is based off two mathematical principles, Euclidean Distance and Centroid.

- Euclidean distance is the distance between two points[1].

- The centroid is the centre of an object or figure[2].

The class has three methods that will be called. The first method is `update_frame()`. 

The methodology behind tracking the cars is this. Firstly, we need to calculate the rectangle based off the countour area detected in the foreground of the video, using background subtraction, as seen in Task/Exercise (1.1), and in here as well. We store all the rectangles in a list of lists and pass it into `update_frame()`. Here, the function will calculate the centroid of the rectangle, which is the car. The centroids are then stored in a separate list, `centroid_history`.

In other words, for every frame in the video, we pass in the rectangle, which is the car, and calculate its centroid.

The next method we call is `track()`. Using the history of detected cars and their centroids,`centroid history`, we calculate euclidean distance between the position of the centroids in the current frame, and the previous frame. This is frame difference technique at work. If the distance is below the threshold, we are tracking the same car, if it is above, it is a new car, and we can update the car counter.  

Lastly, we call `cars_counted()` to get the number of cars counted that are heading towards the city centre, in real time.

In [4]:
class tracking_algorithm():
    def __init__(self):
        #to store all centroid history in current frames and previous frame
        self.centroid_history = []
        self.car_ID_Counter = 0
        
    def get_centroid(self,x, y, w, h):
        cx = x + int(w/2)
        cy = y + int(h/2)
        return cx,cy

    def euclidean_distance(self,x1,x2):
        distance = np.sqrt(np.sum((x1-x2)**2))
        return distance
    
    def update_frame(self,rects):       
        #we want to store the centroids as a list of lists
        singular_object = []
        
        for rect in rects:
            #rects in a list of [x,y,w,h]'s
            x,y,width,height = rect
            
            centroid = self.get_centroid(x,y,width,height)
            singular_object.append(centroid)
        
        self.centroid_history.append(singular_object)
        
        #we only want to have the current frame and previous frame's centroid
        #so if we have length 3, we delete the first elem in the list
        #essentially leaving us with the previous frame[0], and the current frame[1]
        if len(self.centroid_history) == 3:
            self.centroid_history.pop(0)
        
    def track(self):
        #calculate euclidean of current centroid against previous
        if len(self.centroid_history) == 2:    
            curr = self.centroid_history[-1]
            prev = self.centroid_history[0]

            #our list of tuples is store [(x,y)], we access x only
            curr = curr[0][0]
            prev = prev[0][0]
            eucd = self.euclidean_distance(curr,prev)
            
            #we are tracking the same car
            if eucd < 50:
                pass
            #we have found a new car
            else:
                self.car_ID_Counter += 1
    
    #returns counted cars real time
    def cars_counted(self):
        return self.car_ID_Counter

### Main

Here is the main script, similar to Task/Exercise 1.1. Utilizing background subtraction, as well as additional smoothing[3] and morphological transformations[4], which were derived from the OpenCV documentation, which can be viewed here [smoothing techniques](https://docs.opencv.org/3.4/d4/d13/tutorial_py_filtering.html), and here [transformations](https://docs.opencv.org/4.x/d9/d61/tutorial_py_morphological_ops.html).

This is was done to solidify the background subtraction between the cars and Main Street, providing more stable and reliable centroid coordinates, when rectangle of the car's countours is returned.

In [5]:
#read until video is completed or we press 'Q'
def analyze_video(video,video_string):
    '''pass the cv2 video object and directory of the video file'''
    assert isinstance(video_string,str), 'arguement passed in was not a string'
    
    if (video.isOpened()== False): 
        print("Error opening video file")
    
    if video_string == 'Traffic_Laramie_1.mp4':
        print('File Name and Duration:')
        print(video_string,duration1)
    else:
        print('File Name and Duration:')
        print(video_string,duration2)
    
    #detect objects that are being captured
    detect_objs = cv2.createBackgroundSubtractorMOG2(varThreshold=20,detectShadows=False)
    
    #our tracking algorithm
    car_tracker = tracking_algorithm()

    #list to store the number of detected cars
    detected_cars = []
    
    save_prev_frame=0
    car_1_min = None
    car_2_min = None
    
    while True:    
        #capture frame-by-frame
        check, frame = video.read()
        current_frame = video.get(cv2.CAP_PROP_POS_FRAMES)
        #if we reach the end of the video, check will be false
        if not check:
                break
        
        car_counter = car_tracker.cars_counted()
        
        #every new frame, we have an empty list of detected cars, as we do not want to store old frame's car
        detected_cars.clear()
        
        #define 'main street''s dimension
        main_street = frame[300:420,100:820]

        #apply gray and blur to region of interest, main_street
        gray = cv2.cvtColor(main_street,cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5,5), 0 )

        foreground_mask  = detect_objs.apply(blur)

        #to apply additional transformation to frame
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))  
        closing = cv2.morphologyEx(foreground_mask, cv2.MORPH_CLOSE, kernel)
        opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)
        dilation = cv2.dilate(opening,kernel,iterations=2)

        #get contours from the foreground_mask
        contours,*extra = cv2.findContours(foreground_mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

        #rgb colors values
        green = (0,255,0)
        red = (0,0,255)
        
        #draw box to highlight cars and display car counter
        car_counter_display = cv2.putText(frame,'Cars To City Centre:',(10,60),cv2.FONT_HERSHEY_SIMPLEX,1,red,2)
        car_counter_increment = cv2.putText(frame,str(car_counter),(350,60),cv2.FONT_HERSHEY_SIMPLEX,1,red,2)
        
        #draw box and text to label main street
        main_street_start = (0,270)
        main_street_end = (1039,599)
        cv2.rectangle(frame,main_street_start,main_street_end,red,2)
        cv2.putText(frame,'Main Street',(10,260),cv2.FONT_HERSHEY_SIMPLEX,1,red,2)
        
        for cont in contours:
            contour_area = cv2.contourArea(cont)
            if(contour_area>3000):
                #get the cars, push the cords as a list, into detected_cars
                x,y,width,height = cv2.boundingRect(cont)
                geometry = [x,y,width,height]
                detected_cars.append(geometry)
                
                #call function to calculate detected cars
                car_tracker.update_frame(detected_cars)
                car_tracker.track()
                
                #drawing of green box to highlight cars
                cv2.rectangle(main_street,(x,y),(x+width,y+height),green,2)           

               
        td = timedelta(seconds=(current_frame / fps))
        minutes = td.total_seconds() / 60
        
        if minutes == 1.00:
            car_1_min = car_counter
            print('Cars in 1st minute: ',car_counter)
            
        if minutes == 2.00:
            car_2_min = car_counter-car_1_min
            print('Cars in 2nd minute: ',car_2_min)
        
        if check == True:
            cv2.imshow("Main_street_1.2",frame)
            #cv2.imshow("Frame Differencing",dilation)
            
            key = cv2.waitKey(1)
            if key == ord('q'):
                break
            if key == ord('p'):
                cv2.waitKey(-1) #wait until any key is pressed
                
    #after the loop release the video object
    video.release()
    #destroy all the windows
    cv2.destroyAllWindows()
    
    if car_2_min == None:
        print('Cars in final minute:',car_counter-car_1_min)
    else:
        print('Cars in final minute:',car_counter-car_2_min-car_1_min)
    
    print(f'Total number of cars: {car_counter}\n')

In [6]:
analyze_video(video,filepath1)
analyze_video(video2,filepath2)

File Name and Duration:
Traffic_Laramie_1.mp4 2:57
Cars in 1st minute:  2
Cars in 2nd minute:  2
Cars in final minute: 2
Total number of cars: 6

File Name and Duration:
Traffic_Laramie_2.mp4 1:45
Cars in 1st minute:  2
Cars in final minute: 2
Total number of cars: 4



# References

[1] Cuemath. (n.d.). Euclidean Distance Formula - Derivation, Examples. [online] Available at: https://www.cuemath.com/euclidean-distance-formula/.

[2] www.engineeringtoolbox.com. (n.d.). Centroids of Plane Areas. [online] Available at: https://www.engineeringtoolbox.com/centroids-areas-d_2174.html.

[3] docs.opencv.org. (n.d.). OpenCV: Smoothing Images. [online] Available at: https://docs.opencv.org/3.4/d4/d13/tutorial_py_filtering.html.

[4] docs.opencv.org. (n.d.). OpenCV: Morphological Transformations. [online] Available at: https://docs.opencv.org/4.x/d9/d61/tutorial_py_morphological_ops.html.